In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [20]:
train = pd.read_excel('data/train_data.xlsx')
features = train.values[:,:-1]
labels = train.values[:,-1].astype(int)
feature_names = train.columns[:-1]
ss = StandardScaler() # 标准化特征
features = ss.fit_transform(features)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

In [21]:
def accuracy(test_labels, pred_labels):
    correct1 = np.sum(test_labels == pred_labels) # 完全正确
    correct2 = np.sum(abs(test_labels-pred_labels) <= 5) # 近似正确
    n = len(test_labels)
    return [float(correct1)/n, float(correct2)/n]

### 回归用到的四舍五入函数: phrase_x

In [22]:
def phrase_x(x):
    temp = x%5
    if temp>=2.5:
        x = x + 5-temp
    else:
        x = x - temp
    return round(x)

### 回归：XGBRegressor

In [23]:
from xgboost import XGBRegressor

score1 = 0
for train, test in kfold.split(features, labels):
    clf = XGBRegressor(objective='reg:gamma', n_estimators = 100)
    
    val_num = int(len(train)/9)
    train_num = len(train) - val_num
    train_index = train[0:train_num]
    val_index = train[train_num: len(train)+1]
    train_data = features[train_index]
    valid_data = features[val_index]
    train_label = labels[train_index]
    valid_label = labels[val_index]
    clf.fit(train_data, train_label, eval_set = [(valid_data, valid_label)], 
            eval_metric='rmse', early_stopping_rounds = 80, verbose = 200)
    pred = clf.predict(features[test])
    pred1 = [int(phrase_x(x)) for x in pred]
    score = np.array(accuracy(pred1, labels[test]))
    #print(score)
    score1 +=score
print("reg:gamma",score1/10)


print(pd.DataFrame({
        'column': feature_names,
        'importance': clf.feature_importances_,
    }).sort_values(by='importance'))


[0]	validation_0-rmse:26.623
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4.60664
[0]	validation_0-rmse:26.0169
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4.38704
[0]	validation_0-rmse:25.8839
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4.13734
[0]	validation_0-rmse:26.9209
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4.34466
[0]	validation_0-rmse:26.0265
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4.12522
[0]	validation_0-rmse:25.5948
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4.52012
[0]	validation_0-rmse:26.1536
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4.39528
[0]	validation_0-rmse:27.0534
Will train until validation_0-rmse hasn't improved in 80 rounds.
[99]	validation_0-rmse:4

### 回归：LGBMRegressor

In [29]:
from lightgbm import LGBMRegressor

score1 = 0
for train, test in kfold.split(features, labels):
    clf = LGBMRegressor(n_estimators = 100)
    print(train.shape)
    val_num = int(len(train)/9)
    train_num = len(train) - val_num
    train_index = train[0:train_num]
    val_index = train[train_num: len(train)+1]
    train_data = features[train_index]
    valid_data = features[val_index]
    train_label = labels[train_index]
    valid_label = labels[val_index]
    clf.fit(train_data, train_label, eval_set = [(valid_data, valid_label)], early_stopping_rounds =200, verbose = 1000)
    pred = clf.predict(features[test])
    pred1 = [int(phrase_x(x)) for x in pred]
    score = np.array(accuracy(pred1, labels[test]))
    print(score)
    score1 +=score

print("reg:tweedie",score1/10)

(877,)
Training until validation scores don't improve for 200 rounds.
Did not meet early stopping. Best iteration is:
[23]	valid_0's l2: 24.8058
[0.5631068  0.89320388]
(877,)
Training until validation scores don't improve for 200 rounds.
Did not meet early stopping. Best iteration is:
[24]	valid_0's l2: 22.0187
[0.46601942 0.90291262]
(880,)
Training until validation scores don't improve for 200 rounds.
Did not meet early stopping. Best iteration is:
[32]	valid_0's l2: 18.1585
[0.52 0.92]
(881,)
Training until validation scores don't improve for 200 rounds.
Did not meet early stopping. Best iteration is:
[32]	valid_0's l2: 20.9055
[0.49494949 0.90909091]
(882,)
Training until validation scores don't improve for 200 rounds.
Did not meet early stopping. Best iteration is:
[21]	valid_0's l2: 17.2147
[0.52040816 0.89795918]
(882,)
Training until validation scores don't improve for 200 rounds.
Did not meet early stopping. Best iteration is:
[36]	valid_0's l2: 20.8301
[0.5        0.94897959